# Grid Electricity Module

Models grid electricity as originating from a natural gas-fueled simple Brayton cycle. This provides a thermodynamic baseline for upstream exergy consumption.

## Energy Analysis

**Net power output:**
$$\dot{W}_{net} = \dot{W}_t - \dot{W}_c$$

**Fuel energy input (LHV basis):**
$$\dot{Q}_{in} = \dot{m}_f \cdot LHV$$

**First-law efficiency:**
$$\eta_{en} = \frac{\dot{W}_{net}}{\dot{Q}_{in}}$$

Typical range: $\eta_{en} = 0.35 - 0.40$

## Exergy Analysis

**Fuel exergy input:**
$$\dot{E}_f = \dot{m}_f \cdot e_{ch} \approx \dot{m}_f \cdot (\psi \cdot LHV)$$

where $\psi = 1.06$ for natural gas.

**Exergy efficiency:**
$$\eta_{ex} = \frac{\dot{W}_{net}}{\dot{E}_f}$$

**Exergy destruction:**
$$\dot{E}_{d,plant} = \dot{E}_f (1 - \eta_{ex})$$

**Upstream fuel exergy for home electricity:**
$$\dot{E}_{f,up} = \frac{\dot{W}_{home}}{\eta_{ex}}$$

## Parameters

In [1]:
# Grid Electricity Parameters
const GRID_PARAMS = (
    LHV_ng = 50.0e6,        # Lower heating value of natural gas [J/kg]
    ψ_ng = 1.06,            # Chemical exergy factor for natural gas [-]
    η_en_grid = 0.38,       # Energy efficiency of grid power plant [-]
    η_ex_grid = nothing     # Will be calculated
)

(LHV_ng = 5.0e7, ψ_ng = 1.06, η_en_grid = 0.38, η_ex_grid = nothing)

## Core Functions

In [2]:
"""
    GridElectricity

Struct holding grid electricity model parameters.
"""
struct GridElectricity
    LHV::Float64          # Lower heating value [J/kg]
    ψ::Float64            # Chemical exergy factor [-]
    η_en::Float64         # Energy efficiency [-]
    η_ex::Float64         # Exergy efficiency [-]
    
    function GridElectricity(; 
            LHV::Float64 = 50.0e6,
            ψ::Float64 = 1.06,
            η_en::Float64 = 0.38)
        # Exergy efficiency: η_ex = η_en / ψ
        η_ex = η_en / ψ
        new(LHV, ψ, η_en, η_ex)
    end
end

GridElectricity

In [3]:
"""
    fuel_energy_rate(grid::GridElectricity, W_net::Float64) -> Float64

Calculate required fuel energy input rate [W] for given net power output [W].
"""
function fuel_energy_rate(grid::GridElectricity, W_net::Float64)
    return W_net / grid.η_en
end

fuel_energy_rate

In [4]:
"""
    fuel_exergy_rate(grid::GridElectricity, W_net::Float64) -> Float64

Calculate required fuel exergy input rate [W] for given net power output [W].
"""
function fuel_exergy_rate(grid::GridElectricity, W_net::Float64)
    return W_net / grid.η_ex
end

fuel_exergy_rate

In [5]:
"""
    exergy_destruction_rate(grid::GridElectricity, W_net::Float64) -> Float64

Calculate exergy destruction rate [W] in the power plant.
"""
function exergy_destruction_rate(grid::GridElectricity, W_net::Float64)
    E_f = fuel_exergy_rate(grid, W_net)
    return E_f - W_net  # E_d = E_f * (1 - η_ex) = E_f - W_net
end

exergy_destruction_rate

In [6]:
"""
    fuel_mass_rate(grid::GridElectricity, W_net::Float64) -> Float64

Calculate required fuel mass flow rate [kg/s] for given net power output [W].
"""
function fuel_mass_rate(grid::GridElectricity, W_net::Float64)
    Q_in = fuel_energy_rate(grid, W_net)
    return Q_in / grid.LHV
end

fuel_mass_rate

In [7]:
"""
    analyze_grid(grid::GridElectricity, W_home::Float64) -> NamedTuple

Complete analysis for home electricity demand.
Returns all energy and exergy quantities.
"""
function analyze_grid(grid::GridElectricity, W_home::Float64)
    Q_in = fuel_energy_rate(grid, W_home)
    E_f = fuel_exergy_rate(grid, W_home)
    E_d = exergy_destruction_rate(grid, W_home)
    m_f = fuel_mass_rate(grid, W_home)
    
    return (
        W_home = W_home,         # Home electricity demand [W]
        Q_in = Q_in,             # Fuel energy input [W]
        E_f = E_f,               # Fuel exergy input [W]
        E_d = E_d,               # Exergy destruction [W]
        m_f = m_f,               # Fuel mass flow rate [kg/s]
        η_en = grid.η_en,        # Energy efficiency [-]
        η_ex = grid.η_ex         # Exergy efficiency [-]
    )
end

analyze_grid

## Tests

In [8]:
using Test

function run_grid_tests()
    println("Running Grid Electricity Tests...")
    
    # Create default grid model
    grid = GridElectricity()
    
    # Test 1: Parameter validation
    @testset "Parameters" begin
        @test grid.LHV == 50.0e6
        @test grid.ψ == 1.06
        @test grid.η_en == 0.38
        @test grid.η_ex ≈ 0.38 / 1.06 atol=1e-6
    end
    
    # Test 2: Energy balance
    @testset "Energy Balance" begin
        W_home = 5000.0  # 5 kW
        Q_in = fuel_energy_rate(grid, W_home)
        @test Q_in ≈ W_home / grid.η_en atol=1e-6
        @test Q_in > W_home  # Fuel input must exceed output
    end
    
    # Test 3: Exergy balance
    @testset "Exergy Balance" begin
        W_home = 5000.0
        E_f = fuel_exergy_rate(grid, W_home)
        E_d = exergy_destruction_rate(grid, W_home)
        
        # Exergy balance: E_f = W_home + E_d
        @test E_f ≈ W_home + E_d atol=1e-6
        @test E_d > 0  # Must have positive destruction
    end
    
    # Test 4: Mass balance
    @testset "Mass Flow" begin
        W_home = 5000.0
        m_f = fuel_mass_rate(grid, W_home)
        Q_in = fuel_energy_rate(grid, W_home)
        @test m_f ≈ Q_in / grid.LHV atol=1e-10
    end
    
    # Test 5: Full analysis
    @testset "Full Analysis" begin
        W_home = 10000.0  # 10 kW
        result = analyze_grid(grid, W_home)
        
        @test result.W_home == W_home
        @test result.E_f ≈ result.W_home + result.E_d atol=1e-6
        @test result.η_ex < result.η_en  # Exergy efficiency < energy efficiency
    end
    
    println("All tests passed!")
end

run_grid_tests()

Running Grid Electricity Tests...
Test Summary: | Pass  Total  Time
Parameters    |    4      4  0.1s
Test Summary:  | Pass  Total  Time
Energy Balance |    2      2  0.0s
Test Summary:  | Pass  Total  Time
Exergy Balance |    2      2  0.0s
Test Summary: | Pass  Total  Time
Mass Flow     |    1      1  0.0s
Test Summary: | Pass  Total  Time
Full Analysis |    3      3  0.0s
All tests passed!


## Example Usage

In [9]:
# Example: Analyze 5 kW home demand
grid = GridElectricity(η_en = 0.38)
result = analyze_grid(grid, 5000.0)

println("Grid Electricity Analysis")
println("="^40)
println("Home demand:        $(result.W_home/1000) kW")
println("Fuel energy input:  $(round(result.Q_in/1000, digits=2)) kW")
println("Fuel exergy input:  $(round(result.E_f/1000, digits=2)) kW")
println("Exergy destruction: $(round(result.E_d/1000, digits=2)) kW")
println("Fuel mass rate:     $(round(result.m_f*3600, digits=4)) kg/h")
println("Energy efficiency:  $(round(result.η_en*100, digits=1))%")
println("Exergy efficiency:  $(round(result.η_ex*100, digits=1))%")

Grid Electricity Analysis
Home demand:        5.0 kW
Fuel energy input:  13.16 kW
Fuel exergy input:  13.95 kW
Exergy destruction: 8.95 kW
Fuel mass rate:     0.9474 kg/h
Energy efficiency:  38.0%
Exergy efficiency:  35.8%


## Export

Functions available for import:
- `GridElectricity` - Model struct
- `fuel_energy_rate` - Calculate fuel energy input
- `fuel_exergy_rate` - Calculate fuel exergy input  
- `exergy_destruction_rate` - Calculate exergy destruction
- `fuel_mass_rate` - Calculate fuel mass flow
- `analyze_grid` - Complete analysis